In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import numpy as np
import cv2
import os
import imageio.v2 as imageio  # v2 for imwrite consistency


In [53]:
def video_to_array(video_path: str) -> np.ndarray:
    """
    Read all frames from a video into a NumPy array.

    Args:
        video_path: Path to input video file.

    Returns:
        np.ndarray of shape (num_frames, H, W, 3), dtype=uint8 (BGR order).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    frames = []
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frames.append(frame)

    cap.release()
    return np.stack(frames, axis=0)

#output_path = "/data2/saikiran.tedla/hdrvideo/diff/stuttgart_output.mp4"
#video_array = video_to_array(output_path)/255


np_path = "/data2/saikiran.tedla/hdrvideo/diff/stuttgart_output.npy"
video_array = np.load(np_path)  # Load from .npy file if available
video_array = video_array.transpose (1,2,3,0) * 0.5 + 0.5  # Convert from (C,T,H,W) to (T,H,W,C)
video_array = video_array[:,:,:,::-1]  # Convert from RGB to BGR
print("Loaded video shape:", video_array.shape)
#print max and min of video_array
print("Max value in video_array:", np.max(video_array))
print("Min value in video_array:", np.min(video_array))

# #np.mean(np.abs(video_array - quantized_video_array))  # Check difference between two arrays


Loaded video shape: (13, 480, 832, 3)
Max value in video_array: 1.0
Min value in video_array: 0.0


In [54]:
print(f"Video shape: {video_array.shape}, dtype: {video_array.dtype}")

normal_exposure = video_array[1:5] #EV 0
low_exposure = video_array[5:9] #EV -4
high_exposure = video_array[9:13] #EV +4

normal_radiance = normal_exposure
low_radiance = low_exposure * (2 ** 4)  # EV -4
high_radiance = high_exposure * (2 ** -4)  # EV +4

#weight function is a hat function
def weight_function(video):
    """
    Compute weight map for the input video frames.

    Args:
        video: np.ndarray of shape (N, H, W, 3), input video frames.
    Returns:
        np.ndarray of shape (N, H, W, 3), weight maps.
    """
    mid = 0.5
    weight = np.abs(video - mid)
    return weight

def merge_hdr(low_exposure, normal_exposure, high_exposure, low_radiance, normal_radiance, high_radiance):
    """
    Merge low, normal, and high exposure images into an HDR image.

    Args:
        low_exposure: np.ndarray of shape (N, H, W, 3), low exposure images.
        normal_exposure: np.ndarray of shape (N, H, W, 3), normal exposure images.
        high_exposure: np.ndarray of shape (N, H, W, 3), high exposure images.
    Returns:
        np.ndarray of shape (N, H, W, 3)
    """
    w_low = weight_function(low_exposure)
    w_normal = weight_function(normal_exposure)
    w_high = weight_function(high_exposure)

    numerator = (w_low * low_radiance) + (w_normal * normal_radiance) + (w_high * high_radiance)
    denominator = w_low + w_normal + w_high + 1e-8  # Avoid division by zero

    hdr_image = numerator / denominator
    return hdr_image


def output_hdr_video(hdr_video, out_folder):
    os.makedirs(out_folder, exist_ok=True)
    #write out each frame of hdr_video to path as .rad file
    N, H, W, _ = hdr_video.shape
    for i in range(N):
        frame = hdr_video[i]
        filename = f"{out_folder}/frame_{i:04d}.hdr"
        print("Mean of frame:", np.mean(frame))
        cv2.imwrite(filename, frame)  # Save as .hdr image
        
hdr_video = merge_hdr(low_exposure, normal_exposure, high_exposure, low_radiance, normal_radiance, high_radiance)
output_hdr_video(hdr_video, "/data2/saikiran.tedla/hdrvideo/diff/my_hdr_frames")
    




Video shape: (13, 480, 832, 3), dtype: float32
Mean of frame: 0.4598408
Mean of frame: 0.4528929
Mean of frame: 0.5140879
Mean of frame: 0.46188203
